### Design a call center
#### Constraints and assumptions
- What levels of employees are in the call center?
    - Operator, supervisor, director
- Can we assume operators always get the initial calls?
    - Yes
- If there is no available operators or the operator can't handle the call, does the call go to the supervisors?
    - Yes
- If there is no available supervisors or the supervisor can't handle the call, does the call go to the directors?
    - Yes
- Can we assume the directors can handle all calls?
    - Yes
- What happens if nobody can answer the call?
    - It gets queued
- Do we need to handle 'VIP' calls where we put someone to the front of the line?
    - No
- Can we assume inputs are valid or do we have to validate them?
    - Assume they're valid

In [81]:
from abc import ABC, abstractmethod
from enum import Enum
from collections import deque

In [82]:
class Rank(Enum):
    OPERATOR = 0
    SUPERVISOR = 1
    DIRECTOR = 2
    
class CallState(Enum):
    RAEDY = 0
    IN_PROGRESS = 1
    DONE = 2

In [83]:
class Employee(ABC):
    """abstract call center employee"""
    
    def __init__(self, employee_id, name, rank, call_center) -> None:
        self.employee_id = employee_id
        self.name = name
        self.rank = rank
        self.call = None
        self.call_center = call_center
    
    def handle_call(self, call) -> None:
        self.call = call
        call.handler = self
        self.call.state = CallState.IN_PROGRESS
    
    def complete_call(self) -> None:
        self.call.state = CallState.DONE
        self.call_center.notify_call_completed(self.call)
        self.call = None
        
    @abstractmethod
    def escalate_call(self) -> None:
        pass
    
    def _escalate_call(self):
        self.call.state = CallState.READY
        call = self.call
        self.call = None
        self.call_center.notify_call_escalated(call)

In [84]:
class Operator(Employee):
    """concrete operator class"""
    def __init__(self, employee_id, name, call_center):
        super(Operator, self).__init__(emplotee_id, name, Rank.OPERATOR, call_center)
    
    def escalate_call(self):
        self.call.level = Rank.SUPERVISOR
        self._escalate_call()

In [85]:
class Supervisor(Employee):
    """concrete supervisor class"""
    def __init__(self, employee_id, name, call_center):
        super(Supervisor, self).__init__(employee_id, name, Rank.SUPERVISOR, call_center)
    
    def escalate_call(self):
        self.call.level = Rank.DIRECTOR
        self._escalate_call()

In [86]:
class Director(Employee):
    """concrete director class"""
    def __init__(self, employee_id, name, call_center):
        super(Director, self).__init__(employee_id, name, Rank.DIRECTOR, call_center)
    
    def escalate_call(self):
        raise NotImplemented("Director must be able to handle any call")

In [87]:
class Call:
    """concrete call class"""
    def __init__(self, level):
        self.level = Rank.OPERATOR
        self.state = CallState.READY
        self.employee = None

In [88]:
class CallCenter:
    """concrete call center class"""
    
    def __init__(self, operators, supervisors, directors):
        self.operators = operators
        self.supervisors = supervisors
        self.directors = directors
        self.calls_queue = deque()
    
    def dispatch_call(self, call):
        if call.level not in Rank.__members__:
            raise ValueError(f'Invalid all level: {call.level}')
    
        employee = none
    
        if call.level == Rank.OPERATOR:
            employee = self._dispatch_call(call, self.operators)
        if call.level == Rank.SUPERVISOR or employee is None:
            employee = self._dispatch_call(call, self.supervisors)
        if call.level == Rank.DIRECTOR or employee is None:
            employee = self._dispatch_call(call, self.directors)
    
    def _dispatch_call(call, employees):
        for employee in employees:
            if employee.call is None:
                employee.handle_call(call)
                return employee
        return None
    def notify_call_escalated(self, call):pass  # ...
    def notify_call_completed(self, call):pass  # ...
    def dispatch_queued_call_to_newly_freed_employee(self, call, employee):pass  # ...